# 5.7 수업 내용
# 지난 수업 selector + Crawling 

# CSS SELECTOR

In [7]:
import requests
from bs4 import BeautifulSoup

In [2]:
from urllib import parse

header = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}

def getDownload(url, params={}, retries=3):
    resp = None
    
    try:
        resp = requests.get(url, params=params, headers=header)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and retries > 0:
            print(retries)
            resp = getDownload(url, params, retries-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
            print(e.response.headers)
            
    return resp

# 다음

In [37]:
html = getDownload("https://search.daum.net/search", {"q":"박보영"})
daum = BeautifulSoup(html.text, "lxml")

In [12]:
[_["href"] for _ in daum.select("div.wrap_tit > a")]

['http://v.media.daum.net/v/20190507140359069?f=o',
 'http://v.media.daum.net/v/20190507175153358?f=o',
 'http://v.media.daum.net/v/20190507185002021?f=o',
 'http://v.media.daum.net/v/20190507175700537?f=o',
 'http://papa0717.tistory.com/223',
 'http://dudrms606.tistory.com/367',
 'http://adam24eve.tistory.com/858',
 'http://k3k2y.tistory.com/35',
 'https://ko.wikipedia.org/wiki/%EB%B0%95%EB%B3%B4%EC%98%81',
 'https://ko.wikipedia.org/wiki/%EB%B0%95%EB%B3%B4%EC%98%81%20%28%EB%B2%95%EC%A1%B0%EC%9D%B8%29',
 'http://cafe.daum.net/subdued20club/ReHf/2280152?q=%EB%B0%95%EB%B3%B4%EC%98%81',
 'http://cafe.daum.net/subdued20club/ReHf/2282606?q=%EB%B0%95%EB%B3%B4%EC%98%81',
 'http://cafe.daum.net/ASMONACOFC/gAVU/1243818?q=%EB%B0%95%EB%B3%B4%EC%98%81',
 'http://cafe.daum.net/ok1221/9Zdf/1524913?q=%EB%B0%95%EB%B3%B4%EC%98%81',
 'http://tip.daum.net/answer/54503481?q=%EB%B0%95%EB%B3%B4%EC%98%81',
 'http://tip.daum.net/answer/54767339?q=%EB%B0%95%EB%B3%B4%EC%98%81',
 'http://tip.daum.net/answer/550

# 네이버

In [27]:
html = getDownload("https://www.naver.com/")
naver = BeautifulSoup(html.text, "lxml")

In [36]:
[_.get_text() for _ in naver.select('.area_hotkeyword .ah_k ')]

['하나경',
 '울산대교',
 '강은비',
 '한서희',
 '맥반석닭갈비',
 '정용국',
 '고양 창릉',
 '생방송 오늘저녁',
 '3기 신도시',
 '삼성바이오로직스',
 '어비스',
 '이다해',
 '강이준',
 '심재철',
 '부천 대장동',
 '양의지',
 '초면에 사랑합니다',
 '공민규',
 '규현',
 '한선교',
 '하나경',
 '울산대교',
 '강은비',
 '한서희',
 '맥반석닭갈비',
 '정용국',
 '고양 창릉',
 '생방송 오늘저녁',
 '3기 신도시',
 '삼성바이오로직스',
 '어비스',
 '이다해',
 '강이준',
 '심재철',
 '부천 대장동',
 '양의지',
 '초면에 사랑합니다',
 '공민규',
 '규현',
 '한선교']

# Crawling

단어들 인덱스할 것임<br>
scrapy vs beautifulsoup 차이<br>
scrapy는 프레임워크 bs는 라이브러리<br>
페이지 한번 방문할 때 마다 링크를 축적해가며 다시 세팅하여 크롤한다<br>

In [8]:
seed = "http://example.webscraping.com/places/default/index"
html = getDownload(seed)
dom = BeautifulSoup(html.content, 'lxml')

In [9]:
links = [_["href"] for _ in dom.select("a")]
# #이면 자바스크립트 시작기호? 무시해야함
# url 정규화 필요
# 

In [10]:
for link in links:
    # # , /, http, https 가 올 것임
    if link[0] == "/":
        print(requests.compat.urljoin(seed, link))

http://example.webscraping.com/places/default/user/register?_next=/places/default/index
http://example.webscraping.com/places/default/user/login?_next=/places/default/index
http://example.webscraping.com/places/default/index
http://example.webscraping.com/places/default/search
http://example.webscraping.com/places/default/view/Afghanistan-1
http://example.webscraping.com/places/default/view/Aland-Islands-2
http://example.webscraping.com/places/default/view/Albania-3
http://example.webscraping.com/places/default/view/Algeria-4
http://example.webscraping.com/places/default/view/American-Samoa-5
http://example.webscraping.com/places/default/view/Andorra-6
http://example.webscraping.com/places/default/view/Angola-7
http://example.webscraping.com/places/default/view/Anguilla-8
http://example.webscraping.com/places/default/view/Antarctica-9
http://example.webscraping.com/places/default/view/Antigua-and-Barbuda-10
http://example.webscraping.com/places/default/index/1


# url 축적하면서 돌리는 함수

In [89]:
threshold = 3


def urlExtractor(seed, limit):
    if limit > threshold:
        return []
    html = getDownload(seed)
    dom = BeautifulSoup(html.content, "lxml")
    
    links = [_["href"] for _ in dom.select("a") if _.has_attr('href')]
    
    unseen = list()
    
    for link in links:
        if link[0] == "/" and len(link) > 1:
            newLink = requests.compat.urljoin(seed, link)            
            if newLink not in unseen:
                unseen.append((newLink, limit + 1))
            else:
                print("Skipped2", link)
        # 외부로 나가는 주소
        elif link.startswith("http"):
            unseen.append((link, limit + 1))
        else:    
            print("Skipped1", link)
            
    return unseen

In [90]:
urlExtractor(seed)

TypeError: urlExtractor() missing 1 required positional argument: 'limit'

In [46]:
# 처음에 강제실행 한번
queue = urlExtractor(seed, 3)

import time
import random

seen = list()


# 큐가 더이상 없을때까지 돌릴 거
while queue:
    seed = queue.pop(0)
    seen.append(link[0])
#     time.sleep(random.randint(1,3))
    links = urlExtractor(link[0], link[1])
    
    queueURL = [_[0] for _ in queue]
    queue.extend([_ for _ in links if _[0] not in queueURL \
                     and _ not in seen ])
    
    print("{0} {1} [{2}]".format(">"*link[1], link[0], len(links)))
#     html = download(link)
#     dom = BeautifulSoup(html.content, 'lxml')
    # links 는 unseen에 겹치면 안되고 seen에 있으면 안됨
#     queue.extend([_ for _ in links if _ not in queue \
#                   and _ in seen])

InvalidSchema: No connection adapters were found for '('https://ko.wikipedia.org/wiki/%EB%B0%95%EB%B3%B4%EC%98%81_(%EB%8F%99%EC%9D%8C%EC%9D%B4%EC%9D%98)', 4)'

# 구글링크 traversal

In [91]:
import requests
from bs4 import BeautifulSoup

html = getDownload("https://www.google.com/search", {"q":"박보영"})
google = BeautifulSoup(html.text, "lxml")

In [92]:
queue = [(_["href"], 0) for _ in google.select(".r > a")]

In [93]:
# urlExtractor()
# 큐가 더이상 없을때까지 돌릴 거
while queue:
    link = queue.pop(0)
    seen.append(link[0])
    
#     time.sleep(random.randint(1,3))
    print(link)
    links = urlExtractor(link[0], link[1])
    print(links)
    queueURL = [_[0] for _ in queue]
    queue.extend([_ for _ in links if _[0] not in queueURL \
                     and _[0] not in seen ])
    
    print("{0} {1} [{2}]".format(">"+link[1], link[0], len(links)))

('https://ko.wikipedia.org/wiki/%EB%B0%95%EB%B3%B4%EC%98%81', 0)
Skipped1 #mw-head
Skipped1 #p-search
Skipped1 #약력
Skipped1 #성장_과정
Skipped1 #상세_경력
Skipped1 #2006~2008
Skipped1 #2008~2010
Skipped1 #2012~2014
Skipped1 #2015
Skipped1 #2017~현재
Skipped1 #기타
Skipped1 #출연_작품
Skipped1 #수상_및_후보_목록
Skipped1 #각주
Skipped1 #외부_링크
Skipped1 #cite_note-1
Skipped1 #cite_note-2
Skipped1 #cite_note-3
Skipped1 #cite_note-4
Skipped1 #cite_note-5
Skipped1 #cite_note-6
Skipped1 #cite_note-7
Skipped1 #cite_note-8
Skipped1 #cite_note-9
Skipped1 #cite_note-10
Skipped1 #cite_note-11
Skipped1 #cite_note-12
Skipped1 #cite_note-13
Skipped1 #cite_note-14
Skipped1 #cite_note-15
Skipped1 #cite_note-16
Skipped1 #cite_note-17
Skipped1 #cite_note-18
Skipped1 #cite_note-19
Skipped1 #cite_note-20
Skipped1 #cite_note-21
Skipped1 #cite_note-22
Skipped1 #cite_note-23
Skipped1 #cite_note-24
Skipped1 #cite_note-25
Skipped1 #cite_note-26
Skipped1 #cite_note-27
Skipped1 #cite_note-28
Skipped1 #cite_note-29
Skipped1 #cite_note-30


TypeError: must be str, not int

In [64]:
for link in queue:
    html = getDownload(link)
    dom = BeautifulSoup(html.content, 'lxml')
    print(link, len([_["href"] for _ in dom.select("a") if _.has_attr("href")]))

https://ko.wikipedia.org/wiki/%EB%B0%95%EB%B3%B4%EC%98%81 1073
https://ko.wikipedia.org/wiki/%EB%B0%95%EB%B3%B4%EC%98%81%EC%9D%98_%EC%9E%91%ED%92%88_%EB%AA%A9%EB%A1%9D 567
https://namu.wiki/w/%EB%B0%95%EB%B3%B4%EC%98%81 1285
https://www.msn.com/ko-kr/entertainment/news/%EB%B0%95%EB%B3%B4%EC%98%81-tvn-%EB%93%9C%EB%9D%BC%EB%A7%88-%EC%96%B4%EB%B9%84%EC%8A%A4-%EC%97%AC%EC%A3%BC%EC%9D%B8%EA%B3%B5/ar-BBLZzVh 59
https://twitter.com/hashtag/%EB%B0%95%EB%B3%B4%EC%98%81 245
https://www.hankyung.com/entertainment/article/201905030904H 195
https://www.hankyung.com/entertainment/article/201905077647H 189
https://www.mk.co.kr/star/hot-issues/view/2019/05/293931/ 336


In [57]:
requests.compat.urlparse(queue[0])

ParseResult(scheme='https', netloc='ko.wikipedia.org', path='/wiki/%EB%B0%95%EB%B3%B4%EC%98%81', params='', query='', fragment='')

# 네이버 블로그만 크롤링하기

In [56]:
requests.compat.urlparse('https://blog.naver.com/search.naver?where=post&sm=tab_jum&query=%EB%B0%95%EB%B3%B4%EC%98%81')[1]

'blog.naver.com'

In [ ]:
# 큐가 더이상 없을때까지 돌릴 거
while queue:
    seed = queue.pop(0)
    seen.append(link[0])
#     time.sleep(random.randint(1,3))
    links = urlExtractor(link[0], link[1])
    
    queueURL = [_[0] for _ in queue]
    queue.extend([_ for _ in links if _[0] not in queueURL \
                     and _ not in seen ])
    
    print("{0} {1} [{2}]".format(">"+link[1], link[0], len(links)))

In [88]:
html = getDownload("https://arxiv.org/pdf/1708.02709.pdf")
with open("1708.02709.pdf", "wb") as fp:
    fp.write(html.content)

KeyboardInterrupt: 